## Relevant links: 
### https://deepnote.com/@deepnote/3D-network-visualisations-using-plotly-oYxeN6UXSye_3h_ulKV2Dw 
### https://towardsdatascience.com/tutorial-network-visualization-basics-with-networkx-and-plotly-and-a-little-nlp-57c9bbb55bb9
### https://towardsdatascience.com/python-interactive-network-visualization-using-networkx-plotly-and-dash-e44749161ed7 

### https://github.com/jhwang1992/network-visualization/blob/master/app.py 

#### TODO: 
- check the correspondence of the colors and the nodes 
- check if it can/should be directed

## Exploratory Analysis of the dataset

In [734]:
import pandas as pd 
import numpy as np
import networkx as nx
import plotly.graph_objects as go

In [245]:
edges = pd.read_excel (r'raan_case_study interns.xlsx', sheet_name='edges')
nodes=pd.read_excel (r'raan_case_study interns.xlsx', sheet_name='nodes')
nodes=nodes.drop(columns="Unnamed: 3")

In [246]:
edges.head()

,source_id,target_id,weights
0,966,945,13
1,966,879,10
2,649,966,9
3,941,966,8
4,966,467,8


In [845]:
#nodes.head()
nodes.node_color.unique()

array(['#0066CC', '#A05EB5', '#00965E', '#E40046', '#ED8B00', '#B1B3B3'],
      dtype=object)

'#0066CC' blue
#A05EB5   purple
#00965E   green
'#E40046' red
#ED8B00   orange
#B1B3B3   grey

**Observe that we have 2 excel sheets, one containing the edge information 
 (an edge defined between source_id and target_id) and weights
The node sheet contains the node ids, the node colours and the node labels** 

### We have 29 different node_ids, node labels and 6 different color types. 

In [746]:
node_ids=list(set(nodes.node_id.values)) # the unique node_ids that are going to be used for creating the graph
print("the number of unique nodes are:" + str(len(node_ids)))
node_label=list(set(nodes.node_label.values)) 
print("the number of unique label names are:" + str(len(node_label)))
node_colors=list(set(nodes.node_color.values)) 
print("the number of unique color categories are:" + str(len(node_colors)))

the number of unique nodes are:29
the number of unique label names are:29
the number of unique color categories are:6


### Observe that there are some edges (e.g. (879,966) with weight 5 and (966,879) with weight 10). 
#### Therefore, we observe that the direction of the edge makes difference -> use directed graph for the visualization

In [248]:
Gr_dir=nx.from_pandas_edgelist(edges, 'source_id', 'target_id', edge_attr=True, create_using=nx.DiGraph()) #directed graph 

In [67]:
Gr=nx.from_pandas_edgelist(edges, 'source_id', 'target_id', edge_attr=True) #undirected graph, the connections are symmetric 
# less edges than we expect to see otherwise 

In [71]:
Gr.edges

EdgeView([(966, 945), (966, 879), (966, 649), (966, 941), (966, 467), (966, 1042), (966, 785), (966, 619), (966, 457), (966, 639), (966, 747), (966, 185), (966, 349), (966, 1157), (966, 1152), (966, 517), (966, 158), (966, 552), (966, 498), (966, 574), (966, 1025), (966, 813), (966, 792), (966, 172), (966, 1009), (966, 3), (966, 709), (792, 652)])

In [334]:
Gr_dir.edges

OutEdgeView([(966, 945), (966, 879), (966, 467), (966, 1042), (966, 785), (966, 619), (966, 639), (966, 1152), (966, 517), (966, 158), (966, 498), (966, 1157), (966, 185), (966, 1025), (966, 172), (966, 3), (879, 966), (649, 966), (941, 966), (457, 966), (639, 966), (747, 966), (185, 966), (349, 966), (1157, 966), (552, 966), (574, 966), (813, 966), (792, 966), (792, 652), (1009, 966), (709, 966)])

In [337]:
##find the two-way relations between the nodes 
double_edges=[]
for edge in Gr_dir.edges: 
    if Gr_dir.has_edge(edge[1], edge[0]):
        double_edges.append(edge)


In [603]:
double_edges

[(966, 879),
 (966, 639),
 (966, 1157),
 (966, 185),
 (879, 966),
 (639, 966),
 (185, 966),
 (1157, 966)]

#### We observe that not all relations between the nodes are two-way, the network is not symmetric and therefore the direction will play an important role, since it gives information that we don't want to lose. 

In [249]:
atribs=nodes.set_index('node_id').to_dict('index') 
nx.set_node_attributes(Gr_dir, atribs)
#create a dictionary of dictionaries so that we are able to give it as node attributes
# https://stackoverflow.com/questions/54497929/networkx-setting-node-attributes-from-dataframe/54662176
#the keys are the node_ids and the values are a dictionary with keys node_color and node_label 
#access them by e.g. Gr_dir.nodes[3]['node_color']

## Start with 2-d visualization first

In [809]:
#get the positions of the nodes in the graph (different possible layouts, see:
#https://networkx.org/documentation/stable/reference/drawing.html#module-networkx.drawing.layout)
#pos = nx.kamada_kawai_layout(Gr_dir, weight='weights') #dictionary with node names and positions
#pos = nx.random_layout(Gr_dir, seed=2021)
#pos=nx.spring_layout(Gr_dir, weight='weights', seed=2021)
#pos=nx.circular_layout(Gr_dir, scale=2) #dictionary key: node, value: array with coordinates 
pos=nx.shell_layout(Gr_dir)

In [892]:
#pos

In [811]:
pos[966]=np.array([0,0])

In [891]:
#pos

In [813]:
#create a node attribute with the positions of the nodes as found by the layout algorithm
for node in Gr_dir.nodes:
        Gr_dir.nodes[node]['pos'] = list(pos[node]) #has the 2-d positions of the nodes, it is like a node attribute

In [784]:
#nx.get_node_attributes(Gr_dir, 'pos')

In [890]:
def create_edge_trace(edge, pos, is_bidirectional, showlegend=True):
    '''
    edge: a tuple that contains the beginning and the ending of the edge of the graph 
    pos: a dictionary with key the node_id and value the array of 2-d positions of the node
    is bidirectional: True if the edge is bidirectional False otherwise 
    showlegend: True if we want the trace to be shown in the legend
    
    Returns: 
    an edge trace to be used in the 2-d plot
    The opacity and width of the edge are relative to the edge weight
    The bidirected edges are coloured in red while the one-way relations are blue. 
    and the color of the edge 
    '''
    x0, y0 = Gr_dir.nodes[edge[0]]['pos']
    x1, y1 = Gr_dir.nodes[edge[1]]['pos']
    weight = Gr_dir[edge[0]][edge[1]]['weights']
    opacity= weight/np.max(list(nx.get_edge_attributes(Gr_dir,'weights').values())) #normalize between 0 and 1
    if is_bidirectional==True: 
        color='red'
        legendgroup='red'
        name='Two-way Edge'
    else: 
        color='cornflowerblue'
        legendgroup='blue'
        name= 'One-way Edge'
    return go.Scatter(x=tuple([x0, x1, None]), y=tuple([y0, y1, None]),
                        mode='lines',
                        line={'width': 10*weight/np.max(list(nx.get_edge_attributes(Gr_dir,'weights').values())), 'color': color},
                        line_shape='spline',
                        legendgroup=legendgroup,
                        name=name,
                        hoverinfo=None,
                        opacity=opacity, showlegend= showlegend), color
   

In [888]:
edge_trace=[]
edge_col=[]
for edge in Gr_dir.edges:
    if edge in double_edges: 
        if 'red' in edge_col:
            trace,_= create_edge_trace(edge, pos, is_bidirectional = True, showlegend=False)
            edge_trace.append(trace)
        else: 
            trace,col= create_edge_trace(edge, pos, is_bidirectional = True, showlegend=True) 
            edge_col.append(col)
            edge_trace.append(trace)
    else:
        if 'cornflowerblue' in edge_col:
            trace,_= create_edge_trace(edge, pos, is_bidirectional = False, showlegend=False)
            edge_trace.append(trace)
        else:
            trace,col= create_edge_trace(edge, pos, is_bidirectional = False, showlegend=True) 
            edge_col.append(col)
            edge_trace.append(trace)
            

In [787]:
x_node = [pos[i][0] for i in list(nx.get_node_attributes(Gr_dir,'pos').keys())]# x-coordinates of nodes
y_node = [pos[i][1] for i in list(nx.get_node_attributes(Gr_dir,'pos').keys())]# y-coordinates of nodes
#in that way we are sure that the colors and the labels are in the same line with the positions
#we use the same attribute dictionary (the keys are in the same ordering)

In [869]:
def create_node_trace(node, pos, showlegend= True):
    '''
    node: the node id
    pos: the position coordinates of the node in the plot
    showlegend: True if we want to the trace to take part in the legend False otherwise
    
    Returns: 
    trace of the node to be used in the figure
    color of the node 
    '''
    x_node=pos[node][0]
    y_node=pos[node][1]
    color=Gr_dir.nodes[node]['node_color']
    label=Gr_dir.nodes[node]['node_label']
    return go.Scatter(x=[x_node,None],
                      y=[y_node,None],
                      mode='markers',
                      marker=dict(symbol='circle',size=40,color=color),#color the nodes according to their community
                      legendgroup=str(color),
                      name=str(color),
                      showlegend= showlegend,
                      text=label, #label according to the node label
                      hoverinfo='text'), color

In [870]:
node_trace=[]
colors_used=[]
for node in Gr_dir.nodes:
    if Gr_dir.nodes[node]['node_color'] in colors_used: #if already used before don't show it in the legend, just group it with it
        trace,_ = create_node_trace(node, pos, False)
        node_trace.append(trace)
    else: 
        trace, color = create_node_trace(node, pos, True)
        node_trace.append(trace)
        colors_used.append(color)

In [788]:
labels=list(nx.get_node_attributes(Gr_dir,'node_label').values()) #the labels of the nodes 
colors=list(nx.get_node_attributes(Gr_dir,'node_color').values()) #the colors of the nodes

In [855]:
###Create trace for the nodes: 
trace_nodes = go.Scatter(x=x_node,
                         y=y_node,
                        mode='markers',
                        marker=dict(symbol='circle',
                                    size=40,
                                    color=colors),#color the nodes according to their community
                                    text=labels,
                                    hoverinfo='text')

## Create the middle nodes for adding the node annotations

In [863]:
middle_trace = go.Scatter(x=[], y=[], hovertext=[], mode='markers', hoverinfo="text",
                                    marker={'size': 20, 'color': 'LightSkyBlue'},
                                    opacity=0, showlegend= False)


for edge in Gr_dir.edges:
    x0, y0 = Gr_dir.nodes[edge[0]]['pos']
    x1, y1 = Gr_dir.nodes[edge[1]]['pos']
    if edge in double_edges:
        #edge_text='hi'
        edge_text = "Bidirectional edge:" + "<br>" + str(Gr_dir.nodes[edge[0]]['node_label']) + " To: " + str(Gr_dir.nodes[edge[1]]['node_label'])  + ", weight: " + str(Gr_dir.edges[edge]['weights'])+"<br>" + str(Gr_dir.nodes[edge[1]]['node_label']) + " To: " + str(Gr_dir.nodes[edge[0]]['node_label'])  + ", weight: " + str(Gr_dir.edges[(edge[1], edge[0])]['weights'])
    else:
        edge_text="From: " + str(Gr_dir.nodes[edge[0]]['node_label']) + " To: " + str(Gr_dir.nodes[edge[1]]['node_label']) + ", weight: " + str(Gr_dir.edges[edge]['weights'])
    hovertext= edge_text
    middle_trace['x'] += tuple([(x0 + x1) / 2])
    middle_trace['y'] += tuple([(y0 + y1) / 2])
    middle_trace['hovertext'] += tuple([hovertext])
   

In [535]:
layout = go.Layout(
    paper_bgcolor='rgba(0,0,0,0)', # transparent background
    plot_bgcolor='rgba(0,0,0,0)', # transparent 2nd background
    xaxis =  {'showgrid': False, 'zeroline': False}, # no gridlines
    yaxis = {'showgrid': False, 'zeroline': False}, # no gridlines
)

# Create figure
fig = go.Figure(layout = layout)


In [821]:
####NOT USED### 
axis = dict(showbackground=False,
            showline=False,
            zeroline=False,
            showgrid=False,
            showticklabels=False,
            title='')
#also need to create the layout for our plot
layout = go.Layout(title="The network 2-d visualization",
                width=1000,
                height=950,
                showlegend=False,
                scene=dict(xaxis=dict(axis),
                        yaxis=dict(axis)),
                margin=dict(t=100),
                hovermode='closest',
                annotations=[
                            dict(   ax=(Gr_dir.nodes[edge[0]]['pos'][0] + Gr_dir.nodes[edge[1]]['pos'][0]) / 2,
                                    ay=(Gr_dir.nodes[edge[0]]['pos'][1] + Gr_dir.nodes[edge[1]]['pos'][1]) / 2, axref='x', ayref='y',
                                    x=(Gr_dir.nodes[edge[1]]['pos'][0] * 3 + Gr_dir.nodes[edge[0]]['pos'][0]) / 4,
                                    y=(Gr_dir.nodes[edge[1]]['pos'][1] * 3 + Gr_dir.nodes[edge[0]]['pos'][1]) / 4, xref='x', yref='y',
                                    showarrow=True,
                                    arrowhead=3,
                                    arrowsize=5,
                                    arrowwidth=1,
                                    arrowcolor='cornflowerblue',
                                    opacity=0.7
                                ) for edge in Gr_dir.edges])

In [918]:
layout=go.Layout(title='Network 2-d visualization', showlegend=True, hovermode='closest',
                            margin={'b': 60, 'l': 60, 'r': 60, 't': 60},
                            xaxis={'showgrid': False, 'zeroline': False, 'showticklabels': False},
                            yaxis={'showgrid': False, 'zeroline': False, 'showticklabels': False},
                            height=600,
                            #clickmode='closest',
                            annotations=[
                                dict(
                                    ax=(Gr_dir.nodes[edge[0]]['pos'][0] + Gr_dir.nodes[edge[1]]['pos'][0]) / 2,
                                    ay=(Gr_dir.nodes[edge[0]]['pos'][1] + Gr_dir.nodes[edge[1]]['pos'][1]) / 2, axref='x', ayref='y',
                                    x=(Gr_dir.nodes[edge[1]]['pos'][0] * 3 + Gr_dir.nodes[edge[0]]['pos'][0]) / 4,
                                    y=(Gr_dir.nodes[edge[1]]['pos'][1] * 3 + Gr_dir.nodes[edge[0]]['pos'][1]) / 4, xref='x', yref='y',
                                    showarrow=True,
                                    arrowhead=3,
                                    arrowsize=4,
                                    arrowwidth=1,
                                    arrowcolor='red' if edge in double_edges else 'cornflowerblue',
                                    opacity=0.7
                                ) for edge in Gr_dir.edges]
                            )

In [919]:
#data = [edge_trace, trace_nodes]
fig = go.Figure(layout=layout)

for trace in edge_trace:
    fig.add_trace(trace)

for trace in node_trace:
    fig.add_trace(trace)

fig.add_trace(middle_trace)

#fig.update_layout(legend_title_text='Node Types:')
fig.update_layout(legend_itemclick=False)
fig.update_layout(legend_itemdoubleclick=False)
fig.show()
fig.write_html('2d_visualization.html')

### Now try for the 3-d visualization

In [680]:
pos3d = nx.spring_layout(Gr_dir,dim=3, seed=2021, weight='weights')

In [893]:
#pos3d

In [894]:
for node in Gr_dir.nodes:
        Gr_dir.nodes[node]['pos3d'] = list(pos3d[node])

In [291]:
pos3d = nx.kamada_kawai_layout(Gr_dir,dim=3, weight= 'weights')

In [900]:
def create_edge_trace3d(edge, pos, is_bidirectional, showlegend=True):
    '''
    edge: a tuple that contains the beginning and the ending of the edge of the graph 
    pos: a dictionary with key the node_id and value the array of 2-d positions of the node
    is bidirectional: True if the edge is bidirectional False otherwise 
    showlegend: True if we want the trace to be shown in the legend
    
    Returns: 
    an edge trace to be used in the 2-d plot
    The opacity and width of the edge are relative to the edge weight
    The bidirected edges are coloured in red while the one-way relations are blue. 
    and the color of the edge 
    '''
    x0, y0, z0 = Gr_dir.nodes[edge[0]]['pos3d']
    x1, y1, z1 = Gr_dir.nodes[edge[1]]['pos3d']
    weight = Gr_dir[edge[0]][edge[1]]['weights']
    opacity= weight/np.max(list(nx.get_edge_attributes(Gr_dir,'weights').values())) #normalize between 0 and 1
    if is_bidirectional==True: 
        color='red'
        legendgroup='red'
        name='Two-way Edge'
    else: 
        color='cornflowerblue'
        legendgroup='blue'
        name= 'One-way Edge'
    return go.Scatter3d(x=tuple([x0, x1, None]), y=tuple([y0, y1, None]), z=tuple([z0,z1,None]),
                        mode='lines',
                        line={'width': 10*weight/np.max(list(nx.get_edge_attributes(Gr_dir,'weights').values())), 'color': color},
                        legendgroup=legendgroup,
                        name=name,
                        hoverinfo=None,
                        opacity=opacity, showlegend= showlegend), color

In [901]:
edge_trace3d=[]
edge_col=[]
for edge in Gr_dir.edges:
    if edge in double_edges: 
        if 'red' in edge_col:
            trace,_= create_edge_trace3d(edge, pos3d, is_bidirectional = True, showlegend=False)
            edge_trace3d.append(trace)
        else: 
            trace,col= create_edge_trace3d(edge, pos3d, is_bidirectional = True, showlegend=True) 
            edge_col.append(col)
            edge_trace3d.append(trace)
    else:
        if 'cornflowerblue' in edge_col:
            trace,_= create_edge_trace3d(edge, pos3d, is_bidirectional = False, showlegend=False)
            edge_trace3d.append(trace)
        else:
            trace,col= create_edge_trace3d(edge, pos, is_bidirectional = False, showlegend=True) 
            edge_col.append(col)
            edge_trace3d.append(trace)
  

In [903]:
def create_node_trace3d(node, pos, showlegend= True):
    '''
    node: the node id
    pos: the position coordinates of the node in the plot
    showlegend: True if we want to the trace to take part in the legend False otherwise
    
    Returns: 
    trace of the node to be used in the figure
    color of the node 
    '''
    x_node=pos[node][0]
    y_node=pos[node][1]
    z_node=pos[node][2]
    color=Gr_dir.nodes[node]['node_color']
    label=Gr_dir.nodes[node]['node_label']
    return go.Scatter3d(x=[x_node,None],
                      y=[y_node,None],
                      z=[z_node,None],
                      mode='markers',
                      marker=dict(symbol='circle',size=40,color=color),#color the nodes according to their community
                      legendgroup=str(color),
                      name=str(color),
                      showlegend= showlegend,
                      text=label, #label according to the node label
                      hoverinfo='text'), color

In [904]:
node_trace3d=[]
colors_used=[]
for node in Gr_dir.nodes:
    if Gr_dir.nodes[node]['node_color'] in colors_used: #if already used before don't show it in the legend, just group it with it
        trace,_ = create_node_trace3d(node, pos3d, False)
        node_trace3d.append(trace)
    else: 
        trace, color = create_node_trace3d(node, pos3d, True)
        node_trace3d.append(trace)
        colors_used.append(color)

In [906]:
middle_trace3d = go.Scatter3d(x=[], y=[],z=[], hovertext=[], mode='markers', hoverinfo="text",
                                    marker={'size': 20, 'color': 'LightSkyBlue'},
                                    opacity=0, showlegend= False)


for edge in Gr_dir.edges:
    x0, y0,z0 = Gr_dir.nodes[edge[0]]['pos3d']
    x1, y1,z1 = Gr_dir.nodes[edge[1]]['pos3d']
    
    if edge in double_edges:
        #edge_text='hi'
        edge_text = "Bidirectional edge:" + "<br>" + str(Gr_dir.nodes[edge[0]]['node_label']) + " To: " + str(Gr_dir.nodes[edge[1]]['node_label'])  + ", weight: " + str(Gr_dir.edges[edge]['weights'])+"<br>" + str(Gr_dir.nodes[edge[1]]['node_label']) + " To: " + str(Gr_dir.nodes[edge[0]]['node_label'])  + ", weight: " + str(Gr_dir.edges[(edge[1], edge[0])]['weights'])
    else:
        edge_text="From: " + str(Gr_dir.nodes[edge[0]]['node_label']) + " To: " + str(Gr_dir.nodes[edge[1]]['node_label']) + ", weight: " + str(Gr_dir.edges[edge]['weights'])
    hovertext= edge_text
    middle_trace3d['x'] += tuple([(x0 + x1) / 2])
    middle_trace3d['y'] += tuple([(y0 + y1) / 2])
    middle_trace3d['z'] += tuple([(z0 + z1) / 2])
    middle_trace3d['hovertext'] += tuple([hovertext])
   

In [925]:
axis = dict(showbackground=False,
            showline=False,
            zeroline=False,
            showgrid=False,
            showticklabels=False,
            title='')
#also need to create the layout for our plot
layout = go.Layout(title="The network 3-d visualization",
                width=1000,
                height=800,
                #clickmode='event+select',
                showlegend=True,
                scene=dict(xaxis=dict(axis),
                yaxis=dict(axis),
                zaxis=dict(axis)),
                #margin=dict(t=100),
                hovermode='closest')

In [926]:
fig3d = go.Figure(layout=layout)

for trace in edge_trace3d:
    fig3d.add_trace(trace)

for trace in node_trace3d:
    fig3d.add_trace(trace)
    
fig3d.add_trace(middle_trace3d)
fig3d.update_layout(legend_itemclick=False)
fig3d.update_layout(legend_itemdoubleclick=False)
fig3d.show()
fig3d.write_html('3d_visualization.html')